In [81]:
#1 . Write a Python program to connect to MySQL and print the list of all available databases. 
import mysql.connector

In [133]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Revanth123/",
    database="schools_db"  
)
print(conn)

In [134]:
#2. Create a function in Python to create the tables (students, courses, enrollments) in school_db. 
cursor = conn.cursor()
cursor.execute("CREATE DATABASE schools_db")

DatabaseError: 1007 (HY000): Can't create database 'schools_db'; database exists

In [ ]:
cursor.execute("USE schools_db")
cursor.execute("DROP TABLE IF EXISTS enrollments")
cursor.execute("DROP TABLE IF EXISTS courses")
cursor.execute("DROP TABLE IF EXISTS students")

cursor.execute("""
CREATE TABLE students (
    student_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    age INT,
    grade VARCHAR(10)
)""")
print("Table 'students' created.")

cursor.execute("""
CREATE TABLE courses (
    course_id INT AUTO_INCREMENT PRIMARY KEY,
    course_name VARCHAR(100),
    credits INT
)
""")
print(" Table 'courses' created.")

cursor.execute("""
CREATE TABLE enrollments (
    enrollment_id INT AUTO_INCREMENT PRIMARY KEY,
    student_id INT,
    course_id INT,
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
)
""")
print(" Table 'enrollments' created.")

Table 'students' created.
 Table 'courses' created.
 Table 'enrollments' created.


In [ ]:
cursor.execute("SHOW TABLES")
for x in cursor:
    print(x)

('courses',)
('enrollments',)
('students',)


#3. Write a Python program to insert multiple students and courses into the respective tables using executemany().

In [ ]:
import pandas as pd

df_courses = pd.read_csv("./students.csv", usecols=["name", "age","grade"])
vals_students = [tuple(x) for x in df_courses.to_numpy()]

sql_students = "INSERT INTO students (name, age , grade ) VALUES (%s, %s, %s)"
cursor.executemany(sql_students, vals_students)
print(cursor.rowcount, "student records inserted.")

df_courses = pd.read_csv("./courses.csv", usecols=["course_name"])
vals_courses = [tuple(x) for x in df_courses.to_numpy()]

sql_courses = "INSERT INTO courses (course_name) VALUES (%s)"
cursor.executemany(sql_courses, vals_courses)
print(cursor.rowcount, "course records inserted.")

conn.commit()
cursor.close()

100 student records inserted.
10 course records inserted.


In [ ]:
#4 . Fetch and print all rows from the students table in a tabular format. 
cursor.execute("SELECT * FROM students")
rows = cursor.fetchall()                               #discription
columns = [desc[0] for desc in cursor.description]     # extracts only the column names 

df = pd.DataFrame(rows, columns=columns)               #column header 
print("\n Students Table:")
print(df.to_string(index=False))                       

cursor.close()


 Students Table:
 student_id     name  age grade
          1    Zane1   18  None
          2     Uma2   23  None
          3   Wendy3   22  None
          4  Xander4   22  None
          5  Victor5   19  None
          6     Ian6   19  None
          7     Leo7   22  None
          8    Paul8   24  None
          9   Wendy9   23  None
         10    Leo10   23  None
         11    Uma11   18  None
         12   Rita12   18  None
         13   Yara13   22  None
         14    Bob14   19  None
         15 Quincy15   24  None
         16   Tina16   19  None
         17  Alice17   25  None
         18    Sam18   20  None
         19   Paul19   23  None
         20 Victor20   19  None
         21  Frank21   23  None
         22 Victor22   18  None
         23  Karen23   25  None
         24  Grace24   24  None
         25 Quincy25   25  None
         26   Mona26   20  None
         27    Uma27   22  None
         28   Nina28   23  None
         29    Ian29   23  None
         30   Tina30  

Duplicates are removed 

In [ ]:
delete_duplicates = """
DELETE t1 FROM students t1
JOIN students t2 
ON t1.student_id > t2.student_id 
AND t1.name = t2.name 
AND t1.age = t2.age 
AND t1.grade = t2.grade;
"""

cursor.execute(delete_duplicates)
conn.commit()

print("Duplicates removed successfully!")

cursor.close()


Duplicates removed successfully!


In [ ]:
#5 .Write a program to update the grade of a student (given student_id) via a Python input prompt. 
sql = "UPDATE students SET grade = %s WHERE student_id = %s"
val = ("B+", 99)   
cursor.execute(sql, val)
conn.commit()

print(cursor.rowcount, "record updated.")

cursor.close()

0 record updated.


In [ ]:
#6 .Delete a student from the students table using Python input for student_id. 
sql = "DELETE FROM students WHERE name = %s"
val = ("Leo98",)
cursor.execute(sql, val)
conn.commit()

# 7.Write a Python script to insert enrollment records linking students with their courses, ensuring foreign key integrity. 

In [ ]:
cursor.execute("SELECT student_id FROM students")
valid_students = {row[0] for row in cursor.fetchall()}

cursor.execute("SELECT course_id FROM courses")
valid_courses = {row[0] for row in cursor.fetchall()}

valid_enrollments, invalid_enrollments = [], []
with open("enrollments.csv") as f:
    for enroll_id, student_id, course_id in list(csv.reader(f))[1:]:
        student_id, course_id = int(student_id), int(course_id)
        if student_id in valid_students and course_id in valid_courses:
            valid_enrollments.append((enroll_id, student_id, course_id))
        else:
            invalid_enrollments.append((student_id, course_id))

print(f"{len(valid_enrollments)} valid enrollments to insert.")
print(f"{len(invalid_enrollments)} invalid enrollments skipped:", invalid_enrollments[:10])

if valid_enrollments:
    cursor.executemany("INSERT IGNORE INTO enrollments VALUES (%s, %s, %s)", valid_enrollments)
    conn.commit()
    print("Inserted successfully.")

cursor.execute("SELECT * FROM enrollments LIMIT 200")
print(cursor.fetchall())

cursor.close(); 
conn.close()

199 valid enrollments to insert.
0 invalid enrollments skipped: []
Inserted successfully.
[(1, 1, 10), (2, 2, 6), (3, 3, 8), (4, 4, 10), (5, 5, 10), (6, 5, 3), (7, 5, 6), (8, 6, 7), (9, 6, 4), (10, 6, 5), (11, 7, 1), (12, 8, 9), (13, 9, 9), (14, 10, 7), (15, 10, 3), (16, 10, 1), (17, 11, 3), (18, 11, 6), (19, 11, 8), (20, 12, 10), (21, 13, 2), (22, 13, 9), (23, 13, 10), (24, 14, 3), (25, 14, 1), (26, 15, 8), (27, 15, 5), (28, 15, 2), (29, 16, 6), (30, 16, 4), (31, 16, 1), (32, 17, 6), (33, 17, 1), (34, 17, 7), (35, 18, 3), (36, 19, 9), (37, 19, 1), (38, 20, 8), (39, 20, 4), (40, 20, 10), (41, 21, 7), (42, 21, 6), (43, 22, 3), (44, 22, 10), (45, 22, 9), (46, 23, 5), (47, 24, 7), (48, 24, 8), (49, 25, 9), (50, 26, 9), (51, 26, 8), (52, 26, 1), (53, 27, 6), (54, 28, 3), (55, 28, 1), (56, 29, 5), (57, 29, 3), (58, 30, 3), (59, 30, 6), (60, 30, 1), (61, 31, 4), (62, 31, 9), (63, 31, 3), (64, 32, 4), (65, 32, 9), (66, 33, 5), (67, 33, 4), (68, 34, 3), (69, 34, 8), (70, 35, 7), (71, 36, 6), (

8  Fetch all students enrolled in a particular course (given course_id as input).

In [84]:
course_id = int(input("Enter course_id to fetch enrolled students: "))

cursor.execute("""
SELECT s.student_id, s.name, s.age, s.grade, c.course_name
FROM enrollments e
JOIN students s ON e.student_id = s.student_id
JOIN courses c ON e.course_id = c.course_id
WHERE e.course_id = %s
""", (course_id,))

rows = cursor.fetchall()

print(f"\nStudents enrolled in Course {course_id}:")
if rows:
    for row in rows:
        print(row)   # (student_id, name, age, grade, course_name)
else:
    print("No students found for this course.")

cursor.close()
conn.close()


Students enrolled in Course 2:
(13, 'Yara13', 22, 'B', 'Physics')
(15, 'Quincy15', 24, 'A', 'Physics')
(37, 'Ian37', 21, 'D', 'Physics')
(43, 'Quincy43', 25, 'D', 'Physics')
(47, 'Eva47', 19, 'D', 'Physics')
(52, 'Hannah52', 23, 'A', 'Physics')
(53, 'Bob53', 21, 'C', 'Physics')
(57, 'Nina57', 21, 'B', 'Physics')
(60, 'Yara60', 18, 'B', 'Physics')
(75, 'Quincy75', 23, 'D', 'Physics')
(78, 'Ian78', 22, 'D', 'Physics')
(83, 'Leo83', 18, 'C', 'Physics')
(90, 'Zane90', 25, 'D', 'Physics')
(91, 'Tina91', 21, 'D', 'Physics')
(96, 'Ian96', 25, 'C', 'Physics')
(99, 'Bob99', 23, 'C', 'Physics')


9.  Write a program to display students who are not enrolled in any course. 

In [89]:
cursor.execute("""
SELECT s.student_id, s.name, s.age, s.grade
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
WHERE e.student_id IS NULL
""")

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=columns)
print("\nStudents NOT enrolled in any course:")
if df.empty:
    print(" All students are enrolled in some course.")
else:
    print(df.to_string(index=False))

cursor.close()
conn.close()


Students NOT enrolled in any course:
 student_id     name  age grade
        401    Zane1   18     C
        402     Uma2   23     A
        403   Wendy3   22     B
        404  Xander4   22     A
        405  Victor5   19     A
        406     Ian6   19     D
        407     Leo7   22     A
        408    Paul8   24     B
        409   Wendy9   23     B
        410    Leo10   23     A
        411    Uma11   18     C
        412   Rita12   18     B
        413   Yara13   22     B
        414    Bob14   19     C
        415 Quincy15   24     A
        416   Tina16   19     A
        417  Alice17   25     B
        418    Sam18   20     B
        419   Paul19   23     A
        420 Victor20   19     D
        421  Frank21   23     A
        422 Victor22   18     A
        423  Karen23   25     A
        424  Grace24   24     D
        425 Quincy25   25     A
        426   Mona26   20     D
        427    Uma27   22     B
        428   Nina28   23     B
        429    Ian29   23     B
  

10. . Perform an INNER JOIN between students and courses using Python to show 
student names along with the courses they are enrolled in.

In [99]:
cursor.execute("""
SELECT s.student_id, s.name, c.course_name
FROM students s
INNER JOIN enrollments e ON s.student_id = e.student_id
INNER JOIN courses c ON e.course_id = c.course_id
""")

print(" Student Name | Course")
print("-------------------------------------------")
for row in cursor.fetchall():
    print(row)

cursor.close()
conn.close()

 Student Name | Course
-------------------------------------------
(1, 'Zane1', 'Statistics')
(2, 'Uma2', 'Economics')
(3, 'Wendy3', 'English Literature')
(4, 'Xander4', 'Statistics')
(5, 'Victor5', 'Statistics')
(5, 'Victor5', 'Chemistry')
(5, 'Victor5', 'Economics')
(6, 'Ian6', 'Computer Science')
(6, 'Ian6', 'Biology')
(6, 'Ian6', 'History')
(7, 'Leo7', 'Mathematics')
(8, 'Paul8', 'Philosophy')
(9, 'Wendy9', 'Philosophy')
(10, 'Leo10', 'Computer Science')
(10, 'Leo10', 'Chemistry')
(10, 'Leo10', 'Mathematics')
(11, 'Uma11', 'Chemistry')
(11, 'Uma11', 'Economics')
(11, 'Uma11', 'English Literature')
(12, 'Rita12', 'Statistics')
(13, 'Yara13', 'Physics')
(13, 'Yara13', 'Philosophy')
(13, 'Yara13', 'Statistics')
(14, 'Bob14', 'Chemistry')
(14, 'Bob14', 'Mathematics')
(15, 'Quincy15', 'English Literature')
(15, 'Quincy15', 'History')
(15, 'Quincy15', 'Physics')
(16, 'Tina16', 'Economics')
(16, 'Tina16', 'Biology')
(16, 'Tina16', 'Mathematics')
(17, 'Alice17', 'Economics')
(17, 'Alice17'

11. Write a Python function to search for a student by name (partial matches 
allowed using LIKE).

In [103]:
def search_student_by_name(name_part):
 
    sql = "SELECT student_id, name, age, grade FROM students WHERE name LIKE %s"
    val = ("%" + name_part + "%",)  

    cursor.execute(sql, val)
    rows = cursor.fetchall()

    if rows:
        print("Student ID | Name | Age | Grade")
        print("---------------------------------")
        for row in rows:
            print(row)
    else:
        print("No student found with name containing:", name_part)

    cursor.close()
    conn.close()
search_student_by_name("Rita")   

Student ID | Name | Age | Grade
---------------------------------
(12, 'Rita12', 18, 'B')
(62, 'Rita62', 23, 'C')
(85, 'Rita85', 18, 'A')
(412, 'Rita12', 18, 'B')
(462, 'Rita62', 23, 'C')
(485, 'Rita85', 18, 'A')


12.  Add a new column email to the students table using Python and update it 
with random values.

In [106]:
import mysql.connector

cursor.execute("""
UPDATE students s1
JOIN students s2
  ON s1.name = s2.name
SET s1.grade = s2.grade
WHERE s1.student_id BETWEEN 1 AND 100
  AND s2.student_id BETWEEN 101 AND 200;
""")

conn.commit()

cursor.execute("SELECT student_id, name, age, grade, email FROM students WHERE student_id BETWEEN 1 AND 100")
rows = cursor.fetchall()

for row in rows:
    print(row)

cursor.close()
conn.close()


(1, 'Zane1', 18, 'C', 'zane1@school.edu')
(2, 'Uma2', 23, 'A', 'uma2@outlook.com')
(3, 'Wendy3', 22, 'B', 'wendy3@yahoo.com')
(4, 'Xander4', 22, 'A', 'xander4@outlook.com')
(5, 'Victor5', 19, 'A', 'victor5@gmail.com')
(6, 'Ian6', 19, 'D', 'ian6@outlook.com')
(7, 'Leo7', 22, 'A', 'leo7@gmail.com')
(8, 'Paul8', 24, 'B', 'paul8@outlook.com')
(9, 'Wendy9', 23, 'B', 'wendy9@school.edu')
(10, 'Leo10', 23, 'A', 'leo10@yahoo.com')
(11, 'Uma11', 18, 'C', 'uma11@school.edu')
(12, 'Rita12', 18, 'B', 'rita12@outlook.com')
(13, 'Yara13', 22, 'B', 'yara13@yahoo.com')
(14, 'Bob14', 19, 'C', 'bob14@outlook.com')
(15, 'Quincy15', 24, 'A', 'quincy15@school.edu')
(16, 'Tina16', 19, 'A', 'tina16@yahoo.com')
(17, 'Alice17', 25, 'B', 'alice17@outlook.com')
(18, 'Sam18', 20, 'B', 'sam18@gmail.com')
(19, 'Paul19', 23, 'A', 'paul19@school.edu')
(20, 'Victor20', 19, 'D', 'victor20@gmail.com')
(21, 'Frank21', 23, 'A', 'frank21@school.edu')
(22, 'Victor22', 18, 'A', 'victor22@yahoo.com')
(23, 'Karen23', 25, 'A', 


# Delete students with id between 101 and 200

In [110]:
import mysql.connector

# Delete students with id between 101 and 200
cursor.execute("""
DELETE FROM students
WHERE student_id BETWEEN 101 AND 200;
""")

conn.commit()

print(" Duplicate students (IDs 101-200) deleted successfully.")

cursor.execute("SELECT student_id, name, age, grade, email FROM students ORDER BY student_id")
for row in cursor.fetchall():
    print(row)

cursor.close()
conn.close()


 Duplicate students (IDs 101-200) deleted successfully.
(1, 'Zane1', 18, 'C', 'zane1@school.edu')
(2, 'Uma2', 23, 'A', 'uma2@outlook.com')
(3, 'Wendy3', 22, 'B', 'wendy3@yahoo.com')
(4, 'Xander4', 22, 'A', 'xander4@outlook.com')
(5, 'Victor5', 19, 'A', 'victor5@gmail.com')
(6, 'Ian6', 19, 'D', 'ian6@outlook.com')
(7, 'Leo7', 22, 'A', 'leo7@gmail.com')
(8, 'Paul8', 24, 'B', 'paul8@outlook.com')
(9, 'Wendy9', 23, 'B', 'wendy9@school.edu')
(10, 'Leo10', 23, 'A', 'leo10@yahoo.com')
(11, 'Uma11', 18, 'C', 'uma11@school.edu')
(12, 'Rita12', 18, 'B', 'rita12@outlook.com')
(13, 'Yara13', 22, 'B', 'yara13@yahoo.com')
(14, 'Bob14', 19, 'C', 'bob14@outlook.com')
(15, 'Quincy15', 24, 'A', 'quincy15@school.edu')
(16, 'Tina16', 19, 'A', 'tina16@yahoo.com')
(17, 'Alice17', 25, 'B', 'alice17@outlook.com')
(18, 'Sam18', 20, 'B', 'sam18@gmail.com')
(19, 'Paul19', 23, 'A', 'paul19@school.edu')
(20, 'Victor20', 19, 'D', 'victor20@gmail.com')
(21, 'Frank21', 23, 'A', 'frank21@school.edu')
(22, 'Victor22', 

13 .. Create a program to count how many students are enrolled per course and 
display results in descending order

In [114]:
import mysql.connector

cursor.execute("""
SELECT c.course_id, c.course_name, COUNT(e.student_id) AS total_students
FROM courses c
LEFT JOIN enrollments e ON c.course_id = e.course_id
GROUP BY c.course_id, c.course_name
ORDER BY total_students DESC;
""")

print("Course ID | Course Name   | Students Enrolled")
print("-------------------------------------------------")
for row in cursor.fetchall():
    print(f"{row[0]:<9} | {row[1]:<18} | {row[2]}")

cursor.close()
conn.close()


Course ID | Course Name   | Students Enrolled
-------------------------------------------------
3         | Chemistry          | 28
9         | Philosophy         | 22
1         | Mathematics        | 21
5         | History            | 21
7         | Computer Science   | 21
6         | Economics          | 20
10        | Statistics         | 19
4         | Biology            | 18
2         | Physics            | 16
8         | English Literature | 13
11        | Mathematics        | 0
12        | Physics            | 0
13        | Chemistry          | 0
14        | Biology            | 0
15        | History            | 0
16        | Economics          | 0
17        | Computer Science   | 0
18        | English Literature | 0
19        | Philosophy         | 0
20        | Statistics         | 0
21        | Mathematics        | 0
22        | Physics            | 0
23        | Chemistry          | 0
24        | Biology            | 0
25        | History            | 0
26        | Economi

14 .Write a Python script that accepts a course name as input and returns all 
enrolled students with their grades. 

In [117]:
def get_students_by_course(course_name):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Revanth123/",
        database="schools_db"
    )
    cursor = conn.cursor()

    query =("""
        SELECT s.student_id, s.name, s.grade
        FROM students s
        INNER JOIN enrollments e ON s.student_id = e.student_id
        INNER JOIN courses c ON e.course_id = c.course_id
        WHERE c.course_name LIKE %s
    """)
    cursor.execute(query, ("%" + course_name + "%",))

    rows = cursor.fetchall()

    if rows:
        print(f"\nStudents enrolled in {course_name}:")
        print("-------------------------------------")
        print(f"{'ID':<5} | {'Name':<20} | Grade")
        print("-------------------------------------")
        for row in rows:
            print(f"{row[0]:<5} | {row[1]:<20} | {row[2]}")
    else:
        print(f"No students found for course '{course_name}'.")

    cursor.close()
    conn.close()

course_input = input("Enter course name: ")
get_students_by_course(course_input)


Students enrolled in Philosophy:
-------------------------------------
ID    | Name                 | Grade
-------------------------------------
8     | Paul8                | B
9     | Wendy9               | B
13    | Yara13               | B
19    | Paul19               | A
22    | Victor22             | A
25    | Quincy25             | A
26    | Mona26               | D
31    | Nina31               | B
32    | Hannah32             | A
44    | Tina44               | C
54    | Leo54                | C
55    | Yara55               | A
59    | Oscar59              | B
64    | Xander64             | D
72    | Ian72                | D
76    | Oscar76              | D
78    | Ian78                | D
83    | Leo83                | C
87    | Sam87                | C
96    | Ian96                | C
98    | Leo98                | A
100   | Paul100              | B


15. Export the contents of the students table into a CSV file using Python. 

In [ ]:
import csv

cursor.execute("SELECT * FROM students")
rows = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

with open("students_export.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    
    writer.writerow(column_names)
    
    writer.writerows(rows)

print(" Students table exported successfully to students_export.csv")

cursor.close()
conn.close()

 Students table exported successfully to students_export.csv


16. Write a Python program to import student data from a CSV file into the 
students table. 

In [ ]:
with open("./students.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    
    next(reader)
    
    for row in reader:
        student_id, name, age, grade, email = row 
        cursor.execute("""
    INSERT IGNORE INTO students (student_id, name, age, grade, email)
    VALUES (%s, %s, %s, %s, %s)
""", (student_id, name, age, grade, email))
        
conn.commit()

print(" Students data imported successfully from CSV!")

cursor.close()
conn.close()

 Students data imported successfully from CSV!


17 .. Create a stored procedure in MySQL (e.g.,GetStudentCourses(student_id)) and call it from Python. 

In [124]:
student_id = int(input("Enter Student ID: "))

cursor.callproc("GetStudentCourses", [student_id])

for result in cursor.stored_results():
    rows = result.fetchall()
    print("Student ID | Name   | Course    | Grade")
    print("---------------------------------------")
    for row in rows:
        print(f"{row[0]:<10} | {row[1]:<9} | {row[2]:<16} | {row[3]}")
        
cursor.close()
conn.close()

Student ID | Name   | Course    | Grade
---------------------------------------
2          | Uma2      | Economics        | A


C:\Users\16052\AppData\Local\Temp\ipykernel_23428\3846966358.py:5: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for result in cursor.stored_results():


18 .. Handle exceptions in Python to gracefully catch duplicate entries (e.g., 
inserting a student with the same ID). 


In [ ]:
import mysql.connector
from mysql.connector import Error, IntegrityError

try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Revanth123/",
        database="schools_db"
    )
    cursor = conn.cursor()

    student = (1, "Zane1", 18, "A", "zane1@school.edu")   

    try:
        cursor.execute("""
            INSERT INTO students (student_id, name, age, grade, email)
            VALUES (%s, %s, %s, %s, %s)
        """, student)
        conn.commit()
        print(" Student inserted successfully.")

    except IntegrityError as e:     #when violate database rules
        if e.errno == 1062:         #Duplicate primary key
            print(f" Duplicate entry: Student with ID {student[0]} already exists.")
        else:
            print(f" Database error: {e}")

except Error as e:
    print(f" Connection error: {e}")

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print(" MySQL connection closed.")

 Duplicate entry: Student with ID 1 already exists.
 MySQL connection closed.


19. Write a program to implement transactions in Python: Insert a student , Enroll them in a course ,Roll back if the second query fails
   

In [ ]:
import mysql.connector
from mysql.connector import Error

try:
    conn.autocommit = False  

    try:
        student = (101, "NewStudent", 20, "B", "newstudent@example.com")
        cursor.execute("""
            INSERT INTO students (student_id, name, age, grade, email)
            VALUES (%s, %s, %s, %s, %s)
        """, student)

        enrollment = (101, 999)   
        cursor.execute("""
            INSERT INTO enrollments (student_id, course_id)
            VALUES (%s, %s)
        """, enrollment)

        conn.commit()
        print(" Transaction successful: Student inserted & enrolled.")

    except Error as e:
        conn.rollback()
        print(f" Transaction failed, rolled back. Error: {e}")

except Error as e:
    print(f" Connection error: {e}")

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection closed.")

 Transaction failed, rolled back. Error: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`schools_db`.`enrollments`, CONSTRAINT `enrollments_ibfk_2` FOREIGN KEY (`course_id`) REFERENCES `courses` (`course_id`))
MySQL connection closed.


20. Close the database connection safely in Python, ensuring both cursor and connection are properly closed.

In [ ]:
import mysql.connector

try:
    cursor.execute("SELECT * FROM students LIMIT 5;")
    for row in cursor.fetchall():
        print(row)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed safely.")

(1, 'Zane1', 18, 'C', 'zane1@school.edu')
(2, 'Uma2', 23, 'A', 'uma2@outlook.com')
(3, 'Wendy3', 22, 'B', 'wendy3@yahoo.com')
(4, 'Xander4', 22, 'A', 'xander4@outlook.com')
(5, 'Victor5', 19, 'A', 'victor5@gmail.com')
Database connection closed safely.
